## Import data

In [1]:
import os
import pandas as pd
HOUSING_PATH = "datasets/housing"
def load_housing_data():
     housing_path = HOUSING_PATH
     csv_path = os.path.join(housing_path, "housing.csv")
     return pd.read_csv(csv_path)

# All data are stored in housing using panda
housing = load_housing_data()

## Stratified sampling

In [2]:
import numpy as np

# Cap the outlier
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

# Stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
     strat_train_set = housing.loc[train_index]
     strat_test_set = housing.loc[test_index]
    
# Drop "income_cat" attribute    
for set in (strat_train_set, strat_test_set):
     set.drop(["income_cat"], axis=1, inplace=True)

In [3]:
# Make a copy of the data in housing
housing = strat_train_set.copy()

## Data Cleaning

In [4]:
# Remove the predictors, namely "median_house_value"
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [5]:
# Drop text attributes, namely "ocean_proximity", to compute median and apply them to missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)
X = imputer.transform(housing_num)

In [6]:
# Encode the text attribute "ocean_proximity" to binary values.
from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelBinarizer

# Use MyLabelBinarizer to solve further error in the full pipeline
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

housing_cat = housing["ocean_proximity"]
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       ...,
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0]])

In [7]:
# Create a custom transformer to add add_bedrooms_per_room attribute
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
     def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
         self.add_bedrooms_per_room = add_bedrooms_per_room
     def fit(self, X, y=None):
         return self # nothing else to do
     def transform(self, X, y=None):
         rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
         population_per_household = X[:, population_ix] / X[:, household_ix]
         if self.add_bedrooms_per_room: # add_bedrooms_per_room is a helpful attribute
             bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
             return np.c_[X, rooms_per_household, population_per_household,
 bedrooms_per_room]
         else: # add_bedrooms_per_room is NOT a helpful attribute
             return np.c_[X, rooms_per_household, population_per_household]

In [8]:
# Apply the Custom transformer
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [9]:
# Convert Pandas DataFrames to NumPy Array for Scikit-Learn to handle
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
     def __init__(self, attribute_names):
         self.attribute_names = attribute_names
     def fit(self, X, y=None):
         return self
     def transform(self, X):
         return X[self.attribute_names].values

## The whole pipeline to preprocess the data

In [10]:
# The whole pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
num_pipeline = Pipeline([ # only handle numerical attributes
     ('selector', DataFrameSelector(num_attribs)), # convert format
     ('imputer', SimpleImputer(strategy="median")), # take care missing values with median
     ('attribs_adder', CombinedAttributesAdder()), # custom transformer, add add_bedrooms_per_room attribute
     ('std_scaler', StandardScaler()), # Standardize the scale of the features, using Z-score
     ])

cat_pipeline = Pipeline([ # only handle text attributes
     ('selector', DataFrameSelector(cat_attribs)), # convert format
     ('label_binarizer', MyLabelBinarizer()), # tranform text to binary values
     ])

full_pipeline = FeatureUnion(transformer_list=[ # combine the two pipelines together
     ("num_pipeline", num_pipeline),
     ("cat_pipeline", cat_pipeline),
     ])

In [11]:
# Activate the pipeline
housing_prepared=full_pipeline.fit_transform(housing)
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

## Select and Train models

**Random Forest Regressor** found out to work the best, out of **Linear Regression** and **Decision Tree Regressor**

In [12]:
# Train a random forest regressor model
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

In [13]:
# Measure RMSE on Random Forest Regressor on the whole training set
from sklearn.metrics import mean_squared_error
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18663.84658173304

In [14]:
# Cross-validation on Random Forest Regressor into 10 folds
from sklearn.model_selection import cross_val_score
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

In [15]:
# Display the scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(forest_rmse_scores)

Scores: [49318.26453144 47741.04544413 49851.52808726 52001.10011359
 49696.39300753 53046.83420805 48727.70730899 47993.93607886
 52869.856021   49999.17083459]
Mean: 50124.58356354372
Standard deviation: 1808.130116400457


## *Randomized Search

In [18]:
# Randomized Search for different combinations of features
from sklearn.model_selection import RandomizedSearchCV
param_dist = {'n_estimators': range(3,30), 'max_features': range(2,8)}

forest_reg = RandomForestRegressor()

# Cross-validation on all of them, each into 5 folds
randomized_search = RandomizedSearchCV(forest_reg, param_dist, cv=5, scoring='neg_mean_squared_error')

randomized_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'max_features': range(2, 8),
                                        'n_estimators': range(3, 30)},
                   scoring='neg_mean_squared_error')

In [20]:
cvres = randomized_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
randomized_search.best_estimator_

49845.22344590638 {'n_estimators': 28, 'max_features': 6}
51708.57454735382 {'n_estimators': 11, 'max_features': 7}
50679.61124105442 {'n_estimators': 26, 'max_features': 4}
51313.1943653622 {'n_estimators': 14, 'max_features': 7}
52019.64358674851 {'n_estimators': 21, 'max_features': 3}
50110.94894341091 {'n_estimators': 24, 'max_features': 7}
53601.699256494954 {'n_estimators': 20, 'max_features': 2}
50368.223022369406 {'n_estimators': 26, 'max_features': 7}
51694.36048893563 {'n_estimators': 13, 'max_features': 5}
51125.22449792342 {'n_estimators': 19, 'max_features': 4}


RandomForestRegressor(max_features=6, n_estimators=28)

## *Analyze the best model and their errors

In [21]:
# Show the importance of each features
feature_importances = randomized_search.best_estimator_.feature_importances_
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.31572455229481106, 'median_income'),
 (0.1432915485650362, 'INLAND'),
 (0.10329721347771985, 'pop_per_hhold'),
 (0.08457525211440466, 'bedrooms_per_room'),
 (0.08046018699616325, 'longitude'),
 (0.0748723429272985, 'latitude'),
 (0.06290284428457313, 'rooms_per_hhold'),
 (0.04157820490852283, 'housing_median_age'),
 (0.019201433518432034, 'total_rooms'),
 (0.017908350920785087, 'total_bedrooms'),
 (0.0175488667409378, 'population'),
 (0.015951518318327418, 'households'),
 (0.013097344855047491, '<1H OCEAN'),
 (0.005415600826689357, 'NEAR OCEAN'),
 (0.004072633287093761, 'NEAR BAY'),
 (0.00010210596415772989, 'ISLAND')]

## *Evaluate the System on the Test Set

In [22]:
# Take the best model
final_model = randomized_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# Go through the pipeline
X_test_prepared = full_pipeline.transform(X_test)

# Activate predict transform
final_predictions = final_model.predict(X_test_prepared)

# Measure RMSE on the prediction and the actual values
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

final_rmse

47636.79947151527